In [1]:
# 1. Gỡ bỏ hết
!pip uninstall -y transformers sentence-transformers accelerate

# 2. Cài đặt bộ phiên bản "Cổ điển" (Đã test khớp 100%)
!pip install \
    sentence-transformers==2.7.0 \
    transformers==4.44.2 \
    accelerate==0.27.2 \
    "numpy<2.0.0"

Found existing installation: transformers 4.53.3
Uninstalling transformers-4.53.3:
  Successfully uninstalled transformers-4.53.3
Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 82.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.8 MB/s eta 0:00:00:00:010:01
   

In [2]:
import transformers
import accelerate
import sentence_transformers

print(f"✅ Transformers: {transformers.__version__} (Kỳ vọng: 4.44.2)")
print(f"✅ Accelerate: {accelerate.__version__} (Kỳ vọng: 0.27.2)")
print(f"✅ Sentence-Transformers: {sentence_transformers.__version__} (Kỳ vọng: 2.7.0)")

try:
    from sentence_transformers import SentenceTransformer
    # Thử load một model dummy để xem có crash không
    print("🎉 THÀNH CÔNG! Đã import được SentenceTransformer.")
except Exception as e:
    print(f"❌ Vẫn lỗi: {e}")

✅ Transformers: 4.44.2 (Kỳ vọng: 4.44.2)
✅ Accelerate: 0.27.2 (Kỳ vọng: 0.27.2)
✅ Sentence-Transformers: 2.7.0 (Kỳ vọng: 2.7.0)
🎉 THÀNH CÔNG! Đã import được SentenceTransformer.


In [3]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
import pandas as pd
import os

# Tắt chế độ log lên WandB
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

In [4]:
data_dir = "/kaggle/input/uitchatbot"

# Data loading

## Load DataFrame

In [5]:
train_df = pd.read_csv(f"{data_dir}/train.csv")
val_df = pd.read_csv(f"{data_dir}/val.csv")
test_df = pd.read_csv(f"{data_dir}/test.csv")

In [6]:
print("Train data:")
train_df.head()

Train data:


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,index,context,article,document,question,extractive answer,abstractive answer,yes/no
0,8276,Điều 9.\tTuyển bổ sung và loại ra khỏi chương...,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH TÀI NĂNG,Sinh viên dự bị không trở thành sinh viên chín...,02 học kỳ liên tiếp,Sinh viên dự bị không trở thành sinh viên chín...,NaN
1,8543,Điều 4. Kiểm tra xếp lớp đầu khóa cho ...,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,QUY ĐỊNH ĐÀO TẠO NGOẠI NGỮ ĐỐI VỚI HỆ ĐẠI HỌC ...,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...,theo đề xuất hợp lý của bộ phận chuyên môn,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...,NaN
2,7152,Điều 5. Chương trình đào tạo\nCT CLC đ...,Điều 5. Chương trình đào tạo,QUY ĐỊNH ĐÀO TẠO CHƯƠNG TRÌNH CHẤT LƯỢNG CAO,Trình độ tiếng Nhật đạt N mấy mới thì sinh viê...,N3,Sinh viên CT CLC phải đạt tiếng Nhật N3 thì mớ...,NaN
3,2218,Điều 25. Cách tính điểm trung bình\n1. ĐTBH...,Điều 25. Cách tính điểm trung bình,QUY CHẾ ĐÀO TẠO THEO HỌC CHẾ TÍN CHỈ CHO HỆ ĐẠ...,"Cách tính điểm giữa điểm trung bình học kỳ, đi...","ĐTBHK, ĐTBC và ĐTBCTL được tính theo công thức","Cách tính ĐTBHK, ĐTBC và ĐTBCTL được tính theo...",1.0
4,5361,Điều 19. Tổ chức phúc khảo và giải quyết khiế...,Điều 19. Tổ chức phúc khảo và giải quyết khiếu...,QUY ĐỊNH Tổ chức thi các môn học hệ đại học ch...,Hoạt động nào sẽ diễn ra khi SV có khiếu nại v...,phúc khảo,Trường sẽ tổ chức phúc khảo và giải quyết khiế...,NaN


## Load train data

Dữ liệu train có format là `[Câu hỏi, Context đúng]` với mục đích tìm đúng Context cần thiết để trả lời câu hỏi.

In [7]:
train_examples = []
for index, row in train_df.iterrows():
    train_examples.append(InputExample(texts = [row["question"], row["context"]]))

train_dataloader = DataLoader(train_examples, shuffle = True, batch_size = 16)

## Prepare Evaluator

In [8]:
def createEvaluator(df):
    queries = {}    # id câu hỏi -> nội dung câu hỏi
    corpus = {}     # id context -> nội dung context
    relevant_docs = {}  # id câu hỏi -> set các id context đúng

    for index, row in df.iterrows():
        q_id = f"q_{index}"
        c_id = f"c_{index}"

        queries[q_id] = row["question"]
        corpus[c_id] = row["context"]
        relevant_docs[q_id] = {c_id}

    evaluator = evaluation.InformationRetrievalEvaluator(
        queries,
        corpus,
        relevant_docs,
        show_progress_bar = True
    )

    return evaluator

In [9]:
val_evaluator = createEvaluator(val_df)

# Load pre-trained model and evaluate

In [10]:
MODEL_NAME = "hiieu/halong_embedding"

In [11]:
base_model = SentenceTransformer(MODEL_NAME, trust_remote_code=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [12]:
base_results = val_evaluator(base_model)
base_results = val_evaluator.compute_metrices(base_model)
print("Các chỉ số đo được:", base_results.keys())

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:21<00:00, 21.30s/it]


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:22<00:00, 22.68s/it]

Các chỉ số đo được: dict_keys(['cos_sim', 'dot_score'])


# Fine-tune model

In [13]:
train_loss = losses.MultipleNegativesRankingLoss(model = base_model)

In [14]:
num_epochs = 6
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up

In [15]:
print("Starting training...")

base_model.fit(
    train_objectives = [(train_dataloader, train_loss)],
    evaluator = val_evaluator,
    evaluation_steps = 500,
    epochs = num_epochs,
    warmup_steps = warmup_steps,
    output_path = "/kaggle/working/fine_tuned_model",
    save_best_model = True,
    show_progress_bar = True,
    use_amp=True
)

print("Training complete!")

Starting training...


/usr/local/lib/python3.11/dist-packages/sentence_transformers/SentenceTransformer.py:1011: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Iteration:   0%|          | 0/488 [00:00<?, ?it/s]

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

...


Corpus Chunks: 100%|██████████| 1/1 [00:27<00:00, 27.34s/it]


Iteration:   0%|          | 0/488 [00:00<?, ?it/s]

Batches:   0%|          | 0/31 [00:00<?, ?it/s]


Corpus Chunks: 100%|██████████| 1/1 [00:27<00:00, 27.16s/it]


Training complete!


# Compare `base_model` and `fine-tuned model`

In [16]:
test_evaluator = createEvaluator(test_df)

In [17]:
# 1. Vào nhóm Cosine
cosine_metrics = base_results['cos_sim']

# 2. Vào nhóm MRR và Accuracy (nơi chứa các dict con)
mrr_bucket = cosine_metrics['mrr@k']
acc_bucket = cosine_metrics['accuracy@k']

# In thử xem bên trong có những k nào (thường là 1, 3, 5, 10)
print("Các giá trị k có sẵn:", list(mrr_bucket.keys()))

# 3. Lấy giá trị tại k=10 (Lưu ý: số nguyên 10, không phải chuỗi "10")
mrr_10 = mrr_bucket.get(10, 0)
acc_10 = acc_bucket.get(10, 0)

print(f"\n✅ Base Model MRR@10:      {mrr_10:.4f}")
print(f"✅ Base Model Accuracy@10: {acc_10:.4f}")

Các giá trị k có sẵn: [10]

✅ Base Model MRR@10:      0.3334
✅ Base Model Accuracy@10: 0.7254


In [18]:
from sentence_transformers import SentenceTransformer

# 1. Load model đã train
fine_tuned_model = SentenceTransformer("./fine_tuned_model")

# 2. Dùng hàm compute_metrices để lấy bảng kết quả đầy đủ
fine_tuned_results = test_evaluator.compute_metrices(fine_tuned_model)

# 3. Truy cập vào đúng cấu trúc lồng nhau (Nested Dictionary)
# Cấu trúc: {'cos_sim': {'mrr@k': {10: 0.xxx}, 'accuracy@k': {10: 0.xxx}}}
cosine_metrics = fine_tuned_results['cos_sim']
mrr_10 = cosine_metrics['mrr@k'][10]
acc_10 = cosine_metrics['accuracy@k'][10]

# 4. In kết quả
print(f"\n✅ Fine-tuned Model MRR@10:      {mrr_10:.4f}")
print(f"✅ Fine-tuned Model Accuracy@10: {acc_10:.4f}")

You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:27<00:00, 27.61s/it]


✅ Fine-tuned Model MRR@10:      0.4110
✅ Fine-tuned Model Accuracy@10: 0.8381


In [19]:
print("So sánh Base Model và Fine-tuned Model trên tập Test:")

# 1. Lấy chỉ số MRR@10 của Base Model (từ biến base_results đã chạy trước đó)
# Cấu trúc: ['cos_sim']['mrr@k'][10]
base_mrr = base_results['cos_sim']['mrr@k'][10]

# 2. Lấy chỉ số MRR@10 của Fine-tuned Model
# Lưu ý: Phải đảm bảo fine_tuned_results được tạo từ hàm .compute_metrices()
fine_tuned_mrr = fine_tuned_results['cos_sim']['mrr@k'][10]

# 3. Tính độ cải thiện
improvement = fine_tuned_mrr - base_mrr

# 4. In kết quả
print(f"Base Model MRR@10:       {base_mrr:.4f}")
print(f"Fine-tuned Model MRR@10: {fine_tuned_mrr:.4f}")
print(f"---------------------------------------")
print(f"🚀 Mức độ cải thiện:     {improvement:+.4f}")

if improvement > 0:
    print("✅ Chúc mừng! Model fine-tune đã hoạt động hiệu quả hơn.")
else:
    print("⚠️ Model chưa cải thiện. Có thể do tập train quá ít hoặc hyperparameters chưa chuẩn.")

So sánh Base Model và Fine-tuned Model trên tập Test:
Base Model MRR@10:       0.3334
Fine-tuned Model MRR@10: 0.4110
---------------------------------------
🚀 Mức độ cải thiện:     +0.0776
✅ Chúc mừng! Model fine-tune đã hoạt động hiệu quả hơn.


In [29]:
!zip -r /kaggle/working/chroma_db.zip /kaggle/working/chroma_db

  adding: kaggle/working/chroma_db/ (stored 0%)
  adding: kaggle/working/chroma_db/f66da26c-f2c8-4dc8-9e6c-ba5af52fb019/ (stored 0%)
  adding: kaggle/working/chroma_db/f66da26c-f2c8-4dc8-9e6c-ba5af52fb019/data_level0.bin

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 9%)
  adding: kaggle/working/chroma_db/f66da26c-f2c8-4dc8-9e6c-ba5af52fb019/length.bin (deflated 100%)
  adding: kaggle/working/chroma_db/f66da26c-f2c8-4dc8-9e6c-ba5af52fb019/link_lists.bin (deflated 75%)
  adding: kaggle/working/chroma_db/f66da26c-f2c8-4dc8-9e6c-ba5af52fb019/index_metadata.pickle (deflated 44%)
  adding: kaggle/working/chroma_db/f66da26c-f2c8-4dc8-9e6c-ba5af52fb019/header.bin (deflated 54%)
  adding: kaggle/working/chroma_db/chroma.sqlite3 (deflated 49%)


In [20]:
!zip -r /kaggle/working/fine_tuned_model_0_8.zip /kaggle/working/fine_tuned_model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/fine_tuned_model/ (stored 0%)
  adding: kaggle/working/fine_tuned_model/README.md (deflated 55%)
  adding: kaggle/working/fine_tuned_model/model.safetensors (deflated 25%)
  adding: kaggle/working/fine_tuned_model/special_tokens_map.json (deflated 85%)
  adding: kaggle/working/fine_tuned_model/sentence_bert_config.json (deflated 4%)
  adding: kaggle/working/fine_tuned_model/config.json (deflated 50%)
  adding: kaggle/working/fine_tuned_model/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/fine_tuned_model/1_Pooling/ (stored 0%)
  adding: kaggle/working/fine_tuned_model/1_Pooling/config.json (deflated 57%)
  adding: kaggle/working/fine_tuned_model/config_sentence_transformers.json (deflated 35%)
  adding: kaggle/working/fine_tuned_model/modules.json (deflated 62%)
  adding: kaggle/working/fine_tuned_model/2_Normalize/ (stored 0%)
  adding: kaggle/working/fine_tuned_model/eval/ (stored 0%)
  adding: kaggle/working/fine_tuned_model/eval/Information-R

# Re-rank

In [21]:
import os
import sys

# Lệnh cài đặt "Golden Stack" ổn định nhất
if "kaggle_web_client" in sys.modules or "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    print("--- Đang cài đặt thư viện cho Kaggle ---")
    os.system('pip install -q -U langchain==0.1.20 langchain-community==0.0.38 langchain-core==0.1.52 langchain-google-genai==1.0.3 transformers==4.41.2 sentence-transformers==2.7.0 accelerate==0.30.1 chromadb==0.5.0 rank_bm25==0.2.2 "numpy<2.0.0"')
    os.system('pip install -q rank_bm25')
    # Cài đè pandas để fix lỗi numpy binary incompatibility
    os.system('pip install -q "numpy<2.0.0" pandas --force-reinstall')
    os.system('pip install -q rouge-score scikit-learn tqdm')
    print("✅ Cài đặt hoàn tất!")

--- Đang cài đặt thư viện cho Kaggle ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 43.9 MB/s eta 0:00:00
   ━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
google-adk 1.18.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.27.0 which is incompatible.
google-adk 1.18.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.27.0 which is incompatible.
google-adk 1.18.0 requires tenacity<10.0.0,>=9.0.0, but you have tenacity 8.5.0 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-resourcedetector-gcp 1.11.0a0 requires opentelemetry-api~=1.30, but you have opentelemetry-api 1.27.0 which is incompatible.
opentelemetry-resourcedetector-gcp 1.11.0a0 requires 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 kB 26.5 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
google-adk 1.18.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.27.0 which is incompatible.
google-adk 1.18.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.27.0 which is incompatible.
google-adk 1.18.0 requires tenacity<10.0.0,>=9.0.0, but you have tenacity 8.5.0 which is incompatible.
s3fs 2025.3.0 requires fsspec==2025.3.0.*, but you have fsspec 2025.10.0 which is i

✅ Cài đặt hoàn tất!


In [22]:
import shutil
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
from rouge_score import rouge_scorer
import os

# 1. Import Core
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 2. Import Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 3. Import Community (VectorStore & Embeddings)
from langchain_community.vectorstores import Chroma
# Lưu ý: Bản ổn định dùng import này, KHÔNG dùng langchain_huggingface
from langchain_community.embeddings import HuggingFaceEmbeddings

# 4. Import Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI

# 5. Import Retrieval & Re-ranking
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever

In [23]:
FINE_TUNED_MODEL_PATH = "/kaggle/working/fine_tuned_model"
DATA_PATH= "/kaggle/input/uitchatbot"
CHROMA_DB_PATH = "/kaggle/working/chroma_db"

In [24]:
def buildVectorDB(docs):
    """
    Hàm này đọc dữ liệu, vector hóa và lưu vào ChromaDB.
    Chỉ cần chạy 1 lần hoặc khi dữ liệu thay đổi.
    """
    print("--- BẮT ĐẦU XÂY DỰNG DATABASE ---")

    # 1. Kiểm tra model fine-tune
    if not os.path.exists(FINE_TUNED_MODEL_PATH):
        raise FileNotFoundError(f"Không tìm thấy model tại {FINE_TUNED_MODEL_PATH}. Hãy train model trước!")

    # 2. Xóa DB cũ để tránh dữ liệu bị chồng chéo (quan trọng khi đổi model embedding)
    if os.path.exists(CHROMA_DB_PATH):
        print(f"Đang xóa DB cũ tại {CHROMA_DB_PATH}...")
        shutil.rmtree(CHROMA_DB_PATH)

    # 3. Chia nhỏ văn bản (Chunking)
    # Vì context luật thường dài, cần chia nhỏ để model xử lý tốt hơn
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    print(f"Đã chia thành {len(splits)} chunks.")

    # 5. Khởi tạo Embedding Model (Dùng model Fine-tuned)
    print("Đang tải model Embedding Fine-tuned...")
    embedding_model = HuggingFaceEmbeddings(
        model_name=FINE_TUNED_MODEL_PATH,
        model_kwargs={'device': 'cuda'}, # Đổi thành 'cuda' nếu có GPU
        encode_kwargs={'normalize_embeddings': True}
    )

    # 6. Tạo và Lưu Vector Store
    print("Đang tạo Vector Store (ChromaDB)... quá trình này có thể mất vài phút.")
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embedding_model,
        persist_directory=CHROMA_DB_PATH
    )
    vectorstore.persist()
    print("✅ Đã xây dựng xong Vector Database!")
    return vectorstore

In [25]:
print("Đang đọc file CSV...")
df = pd.read_csv(f"{DATA_PATH}/train.csv")
# Lọc bỏ dòng trống
df = df.dropna(subset=['context', 'question'])

# Chuyển đổi sang Document của LangChain
docs = []
for _, row in df.iterrows():
    docs.append(Document(
        page_content=row['context'],
        metadata={
            "source": row.get('document', 'N/A'),
            "article": row.get('article', 'N/A')
        }
    ))

Đang đọc file CSV...


In [26]:
bm25_retriever = BM25Retriever.from_documents(docs) 
bm25_retriever.k = 30  # Lấy 30 doc tốt nhất theo từ khóa

In [28]:
# ... (Các phần import giữ nguyên) ...

# 1. KHỞI TẠO RETRIEVER
print("⚙️ Đang tải Retriever...")

# --- SETUP EMBEDDING (Giữ nguyên) ---
embedding_model = HuggingFaceEmbeddings(
    model_name=FINE_TUNED_MODEL_PATH,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

if not os.path.exists(CHROMA_DB_PATH):
    print("Chưa thấy Database, đang tạo mới...")
    vectorstore = buildVectorDB(docs)
else:
    print("Đã tìm thấy Database, bỏ qua bước tạo mới.")
    vectorstore = Chroma(
        persist_directory=CHROMA_DB_PATH,
        embedding_function=embedding_model
    )

# --- SETUP BASE RETRIEVERS (Giữ nguyên) ---
chroma_retriever = vectorstore.as_retriever(search_kwargs={"k": 20}) # Lấy top 20 thô
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever],
    weights=[0.5, 0.5]
)

# --- THAY ĐỔI QUAN TRỌNG Ở ĐÂY (RERANKER MỚI) ---
print("🚀 Đang load Reranker Model: BAAI/bge-reranker-v2-m3 ...")

# Thay thế ms-marco-MiniLM (tiếng Anh) bằng BGE-M3 (Đa ngôn ngữ/Tiếng Việt)
cross_encoder = HuggingFaceCrossEncoder(
    model_name="BAAI/bge-reranker-v2-m3", 
    model_kwargs={'device': 'cuda'}  # Bắt buộc: Ép chạy bằng GPU để không bị chậm
)

# Lấy top 3 kết quả tốt nhất sau khi rerank
compressor = CrossEncoderReranker(model=cross_encoder, top_n=3)

# Đây là bộ máy tìm kiếm hoàn chỉnh
retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=ensemble_retriever
)

# 2. HÀM ĐÁNH GIÁ (Giữ nguyên logic cũ)
def evaluate_retrieval_only(csv_path):
    print(f"📖 Đang đọc file: {csv_path}")
    df = pd.read_csv(csv_path)
    
    hits = 0            
    mrr_sum = 0         
    total = len(df)
    
    print(f"🚀 Bắt đầu đánh giá {total} câu hỏi (Với BGE-Reranker)...")
    
    for index, row in tqdm(df.iterrows(), total=total):
        question = str(row['question'])
        target_article = str(row['article']).strip().lower()
        
        if not target_article or target_article == "nan":
            total -= 1
            continue

        try:
            # Invoke retriever (Đã bao gồm Rerank)
            retrieved_docs = retriever.invoke(question)
            
            is_found = False
            rank_score = 0
            
            for rank, doc in enumerate(retrieved_docs):
                found_article = str(doc.metadata.get('article', '')).strip().lower()
                
                if target_article in found_article or found_article in target_article:
                    is_found = True
                    rank_score = 1 / (rank + 1)
                    break
            
            if is_found:
                hits += 1
                mrr_sum += rank_score
                
        except Exception as e:
            print(f"Lỗi dòng {index}: {e}")

    if total == 0: total = 1
    hit_rate = hits / total
    mrr_score = mrr_sum / total
    
    print("\n" + "="*40)
    print("📊 KẾT QUẢ ĐÁNH GIÁ (MODEL: BAAI/bge-reranker-v2-m3)")
    print("="*40)
    print(f"✅ Hit Rate (Recall): {hit_rate:.2%} (Mục tiêu: > 85%)")
    print(f"✅ MRR Score:        {mrr_score:.4f} (Mục tiêu: > 0.7)")
    print("="*40)

if __name__ == "__main__":
    evaluate_retrieval_only("/kaggle/input/uitchatbot/test.csv")

You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





⚙️ Đang tải Retriever...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


Đã tìm thấy Database, bỏ qua bước tạo mới.
🚀 Đang load Reranker Model: BAAI/bge-reranker-v2-m3 ...
📖 Đang đọc file: /kaggle/input/uitchatbot/test.csv
🚀 Bắt đầu đánh giá 976 câu hỏi (Với BGE-Reranker)...


100%|██████████| 976/976 [09:20<00:00,  1.74it/s]


📊 KẾT QUẢ ĐÁNH GIÁ (MODEL: BAAI/bge-reranker-v2-m3)
✅ Hit Rate (Recall): 83.91% (Mục tiêu: > 85%)
✅ MRR Score:        0.8033 (Mục tiêu: > 0.7)
